In [70]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign

from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
from langchain.vectorstores import FAISS

from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

from operator import itemgetter

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)


instruct_llm = ChatOpenAI(
  openai_api_key="<REPLACE WITH YOUR OPEN-ROUTER API KEY",
  openai_api_base="https://openrouter.ai/api/v1",
  model_name="mistralai/mistral-7b-instruct:free"
)

embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  This course is about Electricity and Magnetism",
    "[Agent] Electrostatics; Electric Fields; Gauss's Law; Electric Potential; Capacitors and Dielectrics; Current and Resistance; Magnetic Fields and Ampere's Law",
    "[User] Here is my course PDF for Multivariable Calculus",
    "[Agent] Limits and Partial Derivatives; Tangent Planes and Linear Approxiamation; Directional Derivatives and Gradient Vectors; Double Integrals; Triple Integrals; Taylor Polynomials and Polynomial Interpolation; Infinite and Power Series"
    "[User] Textbook for Mechanics"
    "[Agent] Kinematics; Newton's Laws; Work and Energy; Momentum and Impulse; Rotational Motion; Gravitation; Oscillations and Waves" 
]

convstore = FAISS.from_texts(conversation, embedding=embedder)


def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

In [76]:
class question_generator:

    def __init__(self, file_name, description):
        self.description = description

        doc = UnstructuredFileLoader(file_name).load()
        content = json.dumps(doc[0].page_content)
        docs_chunks = [text_splitter.split_documents(doc)]
        docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

        doc_string = "Available Documents:"
        doc_metadata = []

        for chunks in docs_chunks:
            metadata = getattr(chunks[0], 'metadata', {})
            print('METADATA: ')
            print(metadata)
            doc_string += "\n - " + (metadata.get('source').split('.')[0])
            doc_metadata += [str(metadata)]
    
        extra_chunks = [doc_string] + doc_metadata


        vecstores = [FAISS.from_texts(extra_chunks, embedder)]
        vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

        self.docstore = aggregate_vstores(vecstores)

        initial_msg = (
        "Hello! I am a document chat agent here to help the user!"
        f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
        )

        





    def topics_generator(self):
        chat_prompt = ChatPromptTemplate.from_messages([("system",
        "You are an academic TOPIC creator chatbot who divides the course into 7 TOPICS"
        "This is information about the user's course: {input}\n\n"
        " From this, we have retrieved the following potentially-useful info: "
        "Conversation History Retrieval:\n{history}\n\n"
        " Document Retrieval:\n{context}\n\n"
        " (Make sure the topics are only from the sources being used). But don't site the sources"
        "There must be ONLY 7 TOPIC words seperated by semicolons covering the whole course in only one line"
        "No bullet points, lists, paragraphs, sentences, seperate lines, brackets, parentheses, or phrases"
        "Only include the big topics which are course units. Ensure that the topics are in sequential order from least to most difficult"
        "Example Output: 'Kinematics; Newton's Laws; Work and Energy; Momentum and Impulse; Rotational Motion; Gravitation; Oscillations and Waves'"
    ), ('user', '{input}')])
        
        stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

        long_reorder = RunnableLambda(LongContextReorder().transform_documents)

        retrieval_chain = (
        {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
        | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
        | RunnableAssign({'context' : itemgetter('input') | self.docstore.as_retriever()  | long_reorder | docs2str})
        | RPrint()
        )

        def chat_gen(message, history=[], return_buffer=True):
            buffer = ""
    ## First perform the retrieval based on the input message
            retrieval = retrieval_chain.invoke(message)
            line_buffer = ""

    ## Then, stream the results of the stream_chain
            for token in stream_chain.stream(retrieval):
                buffer += token
        ## If you're using standard print, keep line from getting too long
                yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
            save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)
        
        test_question = self.description  ## <- modify as desired

        topics = ""

        for response in chat_gen(test_question, return_buffer=False):
            topics += response
        
        topic_list = topics.split(';')
        print(topic_list, len(topic_list))

In [77]:
topics = question_generator(file_name='ElectricityAndMagnetism.pdf', description = "My course's PDF electricity and magnetism")
topics.topics_generator()

METADATA: 
{'source': 'ElectricityAndMagnetism.pdf'}


{
    'input': "My course's PDF electricity and magnetism",
    'history': "[Quote from Document] [Agent] Electrostatics; Electric Fields; Gauss's Law; Electric Potential; 
Capacitors and Dielectrics; Current and Resistance; Magnetic Fields and Ampere's Law\n[Quote from Document] Agent 
previously responded with Electricity and Magnetism; Maxwell's Equations; Gauss's Law; Electric Fields; Electric 
Potential; Magnet Fields and Ampere's Law; Capacitors and Dielectrics; Light Phenomena and Applications (incl. 
Lightning, Magnetic Levitation, Northern Lights, Sky Color, Rainbows)\n[Quote from Document] [User]  This course is
about Electricity and Magnetism\n[Quote from Document] User previously responded with My course's PDF electricity 
and magnetism\n",
    'context': "[Quote from Document] Available Documents:\n - ElectricityAndMagnetism\n[Quote from Document] 
1\n\nIntroduction\n\nThis course covers electricity and magnetism. At the heart of the topics are the four 
famous\n\nequations known as the Maxwell’s equations,\n\n∇\n\n∇\n\n∇ ×\n\n. ~D = ρ . ~B = 0\n\n~H = ~J +\n\n∂ ~D 
∂t\n\nGauss law (1.1)\n\nNo magnetic monopoles (1.2)\n\nAmpere’s law (1.3)\n\n∇ ×\n\n~E =\n\n−\n\n∂ ~B 
∂t\n\nFaraday’s law (1.4)\n\nAt the end of this course you will understand what each of these equations stand for. 
These equations would lead to the unification of electromagnetism with light and are responsible to explain many 
wonderful phenomena you see in daily life.\n[Quote from Document] {'source': 'ElectricityAndMagnetism.pdf'}\n[Quote
from Document] Warning: This course is extremely math intensive and it would build upon the materials taught during
the lectures. So you have to keep up or else you will be lost. Even sparing a week will put you in trouble. We 
would try to explain phenomena seen in life and connect them to electricity and magnetism. For example, lightening,
magnetic levitation, northern lights, why is the sky blue, rainbows, etc. Our goal is to make you see through these
equations, to make you see and understand the beauty around you and in doing so fell in love with physics. In 
general, the lectures will not be a repeat of your book but will be complementary to the material. The notes are 
accessible online but they do not stand on their own. Please remember that they should be used in combination with 
the lectures.\n\n2 Electrostatics\n\n2.1\n\nIntroduction - What holds the world together?\n"
}

ChatPromptValue(
    messages=[
        SystemMessage(
            content="You are an academic TOPIC creator chatbot who divides the course into 7 TOPICSThis is 
information about the user's course: My course's PDF electricity and magnetism\n\n From this, we have retrieved the
following potentially-useful info: Conversation History Retrieval:\n[Quote from Document] [Agent] Electrostatics; 
Electric Fields; Gauss's Law; Electric Potential; Capacitors and Dielectrics; Current and Resistance; Magnetic 
Fields and Ampere's Law\n[Quote from Document] Agent previously responded with Electricity and Magnetism; Maxwell's
Equations; Gauss's Law; Electric Fields; Electric Potential; Magnet Fields and Ampere's Law; Capacitors and 
Dielectrics; Light Phenomena and Applications (incl. Lightning, Magnetic Levitation, Northern Lights, Sky Color, 
Rainbows)\n[Quote from Document] [User]  This course is about Electricity and Magnetism\n[Quote from Document] User
previously responded with My course's PDF electricity and magnetism\n\n\n Document Retrieval:\n[Quote from 
Document] Available Documents:\n - ElectricityAndMagnetism\n[Quote from Document] 1\n\nIntroduction\n\nThis course 
covers electricity and magnetism. At the heart of the topics are the four famous\n\nequations known as the 
Maxwell’s equations,\n\n∇\n\n∇\n\n∇ ×\n\n. ~D = ρ . ~B = 0\n\n~H = ~J +\n\n∂ ~D ∂t\n\nGauss law (1.1)\n\nNo 
magnetic monopoles (1.2)\n\nAmpere’s law (1.3)\n\n∇ ×\n\n~E =\n\n−\n\n∂ ~B ∂t\n\nFaraday’s law (1.4)\n\nAt the end 
of this course you will understand what each of these equations stand for. These equations would lead to the 
unification of electromagnetism with light and are responsible to explain many wonderful phenomena you see in daily
life.\n[Quote from Document] {'source': 'ElectricityAndMagnetism.pdf'}\n[Quote from Document] Warning: This course 
is extremely math intensive and it would build upon the materials taught during the lectures. So you have to keep 
up or else you will be lost. Even sparing a week will put you in trouble. We would try to explain phenomena seen in
life and connect them to electricity and magnetism. For example, lightening, magnetic levitation, northern lights, 
why is the sky blue, rainbows, etc. Our goal is to make you see through these equations, to make you see and 
understand the beauty around you and in doing so fell in love with physics. In general, the lectures will not be a 
repeat of your book but will be complementary to the material. The notes are accessible online but they do not 
stand on their own. Please remember that they should be used in combination with the lectures.\n\n2 
Electrostatics\n\n2.1\n\nIntroduction - What holds the world together?\n\n\n (Make sure the topics are only from 
the sources being used). But don't site the sourcesThere must be ONLY 7 TOPIC words seperated by semicolons 
covering the whole course in only one lineNo bullet points, lists, paragraphs, sentences, seperate lines, brackets,
parentheses, or phrasesOnly include the big topics which are course units. Ensure that the topics are in sequential
order from least to most difficultExample Output: 'Kinematics; Newton's Laws; Work and Energy; Momentum and 
Impulse; Rotational Motion; Gravitation; Oscillations and Waves'",
            additional_kwargs={},
            response_metadata={}
        ),
        HumanMessage(
            content="My course's PDF electricity and magnetism",
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

[' Electrostatics', " Maxwell's Equations", ' Electric Fields', " Gauss's Law", ' Electric Potential', " Magnetic Fields and Ampere's Law", ' Capacitors and Dielectrics', ' Light Phenomena (including Lightning, Magnetic Levitation, Northern Lights, Sky Color, Rainbows)'] 8


In [78]:
calculusTopics = question_generator(file_name='MultivariableCalculus.pdf', description = "My course's textbook is about Calculus II")
calculusTopics.topics_generator()

METADATA: 
{'source': 'MultivariableCalculus.pdf'}


{
    'input': "My course's textbook is about Calculus II",
    'history': "[Quote from Document] [Agent] Limits and Partial Derivatives; Tangent Planes and Linear 
Approxiamation; Directional Derivatives and Gradient Vectors; Double Integrals; Triple Integrals; Taylor 
Polynomials and Polynomial Interpolation; Infinite and Power Series[User] Textbook for Mechanics[Agent] Kinematics;
Newton's Laws; Work and Energy; Momentum and Impulse; Rotational Motion; Gravitation; Oscillations and 
Waves\n[Quote from Document] [User]  This course is about Electricity and Magnetism\n[Quote from Document] [Agent] 
Electrostatics; Electric Fields; Gauss's Law; Electric Potential; Capacitors and Dielectrics; Current and 
Resistance; Magnetic Fields and Ampere's Law\n[Quote from Document] [User] Here is my course PDF for Multivariable 
Calculus\n",
    'context': '[Quote from Document] Math 119\n\nCalculus II for Engineering\n\nLecture Notes\n\nDavid 
Harmsworth\n\nDepartment of Applied Mathematics\n\nUniversity of Waterloo\n\n2014, 2018\n\nPart I\n\nMultivariate 
Calculus\n\n1\n\nIntroduction\n\nIn Math 117 we discussed the essentials of the calculus of functions of one 
variable, and\n\nrelationships between variables deﬁned by those functions: y = f (x), 
or\n\nx\n\n→\n\nf\n\n→\n\ny.\n\nWe could also write f : R\n\n→\n\nR, of course, to denote that f takes one (real) 
input and gives\n\none (real) output. Although, as we’ve seen, there are lots of things we can do with 
these\n\nfunctions, many real-world phenomena depend on multiple variables, and so in the ﬁrst half of\n\nMath 119,
we will turn our attention to functions which take multiple variables as input. As\n\na couple of simple motivating
examples, consider that the altitude of the earth’s surface could\n\nbe viewed as a function of both latitude and 
longitude, while the temperature in a region of\n[Quote from Document] π 4\n\nˆ 0 ˆ 1\n\nsec φ\n\nρdρdφ =\n\nˆ 
0\n\nπ 4\n\nρ2 2\n\n(cid:12) sec φ (cid:12) (cid:12) (cid:12) 1\n\ndφ\n\n=\n\n1 2 ˆ 0\n\nπ 4\n\n(cid:0)sec2 
φ\n\n−\n\n1(cid:1) dφ\n\n=\n\n1 2\n\n(tan φ\n\n−\n\n(cid:12) (cid:12) φ) (cid:12) (cid:12)\n\nπ 4\n\n0\n\n=\n\n1 2 
−\n\nπ 8\n\n.\n\nYou’ll see another example or two in the assignments.\n\nMean Values\n\nHere’s one other special 
application of the deﬁnite integral; it can be used to calculate average\n\nvalues of functions over speciﬁed 
domains. We discussed the single-variable case in Math 117,\n\nand showed that the average value of a function f 
(x) over an interval [a, b] can be calculated\n\nas\n\nfavg =\n\nb\n\n1\n\n−\n\na ˆ a\n\nb\n\nf 
(x)dx.\n\nSimilarly, the average value of a function f (x, y) over a two-dimensional region R can be\n\ncalculated 
as\n\nfavg =\n\n1\n\nArea(R) ¨R\n\nf (x, y)dA.\n\n64\n\nExample: Find the average distance from the origin of all 
the points within the unit circle.\n\nSolution: The distance of a point (x, y) from the origin is given by f (x, y)
=\n\n(cid:112)\n\nand so\n\nR Average Distance = ˜\n[Quote from Document] to the variable of integration; it may 
depend upon the other variable(s), and unless we know\n\notherwise we must write it as 
such.\n\nExample:\n\n´\n\n(x2 + y2) dy = x2y +\n\ny3 3\n\n+ g(x).\n\nWe can use this idea to identify a function 
(up to a constant) when we know both of its\n\npartial derivatives.\n\nExample: Suppose we know that fx = 6xy2 + 
ey, and fy = 6x2y + xey + sin y. Then we\n\nmay write\n\nf (x, y) =\n\nˆ\n\nfxdx =\n\nˆ\n\n(cid:0)6xy2 + ey(cid:1) 
dx = 3x2y2 + xey + g (y) .\n\nAt the same time, we also know that\n\nf (x, y) =\n\nˆ\n\nfydy =\n\nˆ\n\n(cid:0)6x2y 
+ xey + sin y(cid:1) dy = 3x2y2 + xey\n\n−\n\ncos y + h (x) .\n\nComparing the two expressions, we can conclude 
that f (x, y) = 3x2y2 + xey\n\n−\n\ncos y + C, for\n\nsome constant C.\n\nThis is straightforward enough, but it 
isn’t really integration; it’s just antidiﬀerentiation.\n\nThese are not the same thing! They just happen to be 
related to each other - in single-variable\n\ncalc

ChatPromptValue(
    messages=[
        SystemMessage(
            content="You are an academic TOPIC creator chatbot who divides the course into 7 TOPICSThis is 
information about the user's course: My course's textbook is about Calculus II\n\n From this, we have retrieved the
following potentially-useful info: Conversation History Retrieval:\n[Quote from Document] [Agent] Limits and 
Partial Derivatives; Tangent Planes and Linear Approxiamation; Directional Derivatives and Gradient Vectors; Double
Integrals; Triple Integrals; Taylor Polynomials and Polynomial Interpolation; Infinite and Power Series[User] 
Textbook for Mechanics[Agent] Kinematics; Newton's Laws; Work and Energy; Momentum and Impulse; Rotational Motion; 
Gravitation; Oscillations and Waves\n[Quote from Document] [User]  This course is about Electricity and 
Magnetism\n[Quote from Document] [Agent] Electrostatics; Electric Fields; Gauss's Law; Electric Potential; 
Capacitors and Dielectrics; Current and Resistance; Magnetic Fields and Ampere's Law\n[Quote from Document] [User] 
Here is my course PDF for Multivariable Calculus\n\n\n Document Retrieval:\n[Quote from Document] Math 
119\n\nCalculus II for Engineering\n\nLecture Notes\n\nDavid Harmsworth\n\nDepartment of Applied 
Mathematics\n\nUniversity of Waterloo\n\n2014, 2018\n\nPart I\n\nMultivariate Calculus\n\n1\n\nIntroduction\n\nIn 
Math 117 we discussed the essentials of the calculus of functions of one variable, and\n\nrelationships between 
variables deﬁned by those functions: y = f (x), or\n\nx\n\n→\n\nf\n\n→\n\ny.\n\nWe could also write f : 
R\n\n→\n\nR, of course, to denote that f takes one (real) input and gives\n\none (real) output. Although, as we’ve 
seen, there are lots of things we can do with these\n\nfunctions, many real-world phenomena depend on multiple 
variables, and so in the ﬁrst half of\n\nMath 119, we will turn our attention to functions which take multiple 
variables as input. As\n\na couple of simple motivating examples, consider that the altitude of the earth’s surface
could\n\nbe viewed as a function of both latitude and longitude, while the temperature in a region of\n[Quote from 
Document] π 4\n\nˆ 0 ˆ 1\n\nsec φ\n\nρdρdφ =\n\nˆ 0\n\nπ 4\n\nρ2 2\n\n(cid:12) sec φ (cid:12) (cid:12) (cid:12) 
1\n\ndφ\n\n=\n\n1 2 ˆ 0\n\nπ 4\n\n(cid:0)sec2 φ\n\n−\n\n1(cid:1) dφ\n\n=\n\n1 2\n\n(tan φ\n\n−\n\n(cid:12) (cid:12)
φ) (cid:12) (cid:12)\n\nπ 4\n\n0\n\n=\n\n1 2 −\n\nπ 8\n\n.\n\nYou’ll see another example or two in the 
assignments.\n\nMean Values\n\nHere’s one other special application of the deﬁnite integral; it can be used to 
calculate average\n\nvalues of functions over speciﬁed domains. We discussed the single-variable case in Math 
117,\n\nand showed that the average value of a function f (x) over an interval [a, b] can be 
calculated\n\nas\n\nfavg =\n\nb\n\n1\n\n−\n\na ˆ a\n\nb\n\nf (x)dx.\n\nSimilarly, the average value of a function f
(x, y) over a two-dimensional region R can be\n\ncalculated as\n\nfavg =\n\n1\n\nArea(R) ¨R\n\nf (x, 
y)dA.\n\n64\n\nExample: Find the average distance from the origin of all the points within the unit 
circle.\n\nSolution: The distance of a point (x, y) from the origin is given by f (x, y) =\n\n(cid:112)\n\nand 
so\n\nR Average Distance = ˜\n[Quote from Document] to the variable of integration; it may depend upon the other 
variable(s), and unless we know\n\notherwise we must write it as such.\n\nExample:\n\n´\n\n(x2 + y2) dy = x2y 
+\n\ny3 3\n\n+ g(x).\n\nWe can use this idea to identify a function (up to a constant) when we know both of 
its\n\npartial derivatives.\n\nExample: Suppose we know that fx = 6xy2 + ey, and fy = 6x2y + xey + sin y. Then 
we\n\nmay write\n\nf (x, y) =\n\nˆ\n\nfxdx =\n\nˆ\n\n(cid:0)6xy2 + ey(cid:1) dx = 3x2y2 + xey + g (y) .\n\nAt the 
same time, we also know that\n\nf (x, y) =\n\nˆ\n\nfydy =\n\nˆ\n\n(cid:0)6x2y + xey + sin y(cid:1) dy = 3x2y2 + 
xey\n\n−\n\ncos y + h (x) .\n\nComparing the two expressions, we can conclude

[' Limits', ' Partial Derivatives', ' Tangent Planes and Linear Approximation', ' Directional Derivatives and Gradient Vectors', ' Multivariate Integration', ' Taylor Polynomials and Polynomial Interpolation', ' Infinite and Power Series'] 7
